In [3]:
import numpy as np
import pandas as pd
from pathlib import Path
from tqdm.auto import tqdm

In [48]:
from icesea2.experiments import gen_lhs_from_dict, norm_to_values, topography_to_icesheet_nc, gen_run_configs
from icesea2.margins import shp_series_to_icesheet
from icesea2.shearstress import make_shear_stress_map
from icesea2.models import IcesheetModel

In [49]:
times = [-140000]

In [50]:
shear_stress_parameters_var_ranges = {
    "g_pgm_ice_streams_ice_stream": [5000,20000],
    "g_sediment_marine_sediment": [10000,30000],
    "g_sediment_onshore_sediment": [30000,100000],
    "g_sediment_bedrock": [100000,150000],
    "p_ice_stream_interior_dist": [0,1000],
    "p_cold_ice_shear_stress": [120000,200000],
    "p_cold_ice_interior_dist": [1000,300],
    "p_hybrid_ice_stream_shear_stress": [5000,100000],
    "p_hybrid_ice_stream_dist": [0,600],
}

In [51]:
lhs_norm_path = "lhs_norm.csv"
lhs_values_path = "lhs_values.csv"

lhs = np.genfromtxt(lhs_norm_path, delimiter=",", skip_header=1)
lhs_values = np.genfromtxt(lhs_values_path, delimiter=",", skip_header=1)

exp_vars = pd.DataFrame(lhs_values, columns=list(shear_stress_parameters_var_ranges.keys()))

In [53]:
run_dir = "/nfs/annie/cm15ogp/phd/paper1/paper1_data/icesheet_runs/batchelor/topo_iteration_1"

In [54]:
t_var_topo = True
e_var_topo = True
e_var_ss = True
t_var_ss = False
e_var_margins = False

In [55]:
run_configs = []
for member, config in exp_vars.iterrows():
    config = config.to_dict()
    run_config = gen_run_configs(
        run_dir = str(run_dir),
        times = times,
        member_name = str(member),
        t_var_topo=t_var_topo,
        e_var_topo=e_var_topo,
        t_var_ss=t_var_ss,
        e_var_ss=e_var_ss,
        e_var_margins=e_var_margins
    )
    run_configs = run_configs + run_config

In [56]:
ism = IcesheetModel("/nfs/annie/cm15ogp/code/icesheet/binaries/icesheet")
ism.run_pool(run_configs, cores=20)